# Avaliação de Desempenho

## Estudo de caso: Acidentes de Trânsito na cidade de Porto Alegre/RS
Este exemplo usa a base de dados abaixo, sobre acidentes de trânsito na cidade de Porto Alegre, para criar modelos preditivos de classificação para prever se haverá ou não morte de alguma vítima hospitalizada em até 30 dias após o acidente (coluna alvo morte_post).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('cat_acidentes.csv',sep=';')
df.head(3)

,data_extracao,idacidente,longitude,latitude,log1,log2,predial1,tipo_acid,queda_arr,data,...,caminhao,moto,carroca,bicicleta,outro,noite_dia,regiao,cont_vit,ups,consorcio
0,2020-04-01 01:33:32,600976,-51.235877,-30.030187,R GEN BENTO MARTINS,R SIQUEIRA CAMPOS,0,ABALROAMENTO,0,2015-01-01 00:00:00,...,0,0,0,0,0,NOITE,CENTRO,1,5,NaN
1,2020-04-01 01:33:32,601032,-51.214623,-30.043469,NaN,NaN,0,ABALROAMENTO,0,2015-01-03 00:00:00,...,0,0,0,0,0,NOITE,CENTRO,0,1,NaN
2,2020-04-01 01:33:32,601045,-51.158916,-30.063865,AV BENTO GONCALVES,NaN,5690,ABALROAMENTO,0,2015-01-05 00:00:00,...,0,0,0,0,0,DIA,LESTE,1,5,NaN


## Pré-processamento

In [2]:
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing   import StandardScaler
from sklearn.preprocessing   import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline        import Pipeline
from sklearn.compose         import ColumnTransformer

# atributos a serem removidos: data_extracao, idacidente, log1, log2, predial1, 
#                              data, hora, consorcio, noite_dia, cont_vit
nomes_atributos_numericos   = ['longitude','latitude','feridos','feridos_gr','mortes','auto','taxi','lotacao','onibus_urb','onibus_met',
                               'onibus_int','caminhao','moto','carroca','bicicleta','outro']
nomes_atributos_binarios    = ['queda_arr','dia_bin']
nomes_atributos_categoricos = ['tipo_acid','dia_sem','regiao','ups']
nome_coluna_alvo            = ['morte_post']

# Preenchendo dados ausentes na coluna noite_dia
imputer = SimpleImputer(strategy='most_frequent')
df['noite_dia'] = imputer.fit_transform(df[['noite_dia']])

# Transformando noite_dia em uma única coluna binária
df['dia_bin'] = 0
df.loc[df['noite_dia']=='DIA','dia_bin'] = 1

# Remoção de linhas com valores ausentes
df = df.dropna(subset=['longitude','latitude'])

# separando os conjuntos de dados de treino e teste
df_treino, df_teste = train_test_split(df, test_size=0.2, random_state=42)

# separando a coluna alvo do conjunto de treino
df_treino_labels = df_treino[nome_coluna_alvo].copy()
df_treino        = df_treino.drop(columns=nome_coluna_alvo)

# separando a coluna alvo do conjunto de teste
df_teste_labels = df_teste[nome_coluna_alvo].copy()
df_teste        = df_teste.drop(columns=nome_coluna_alvo)

# Pipelines de Pré-processamento
pipeline_atr_numericos = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos',   pipeline_atr_numericos, nomes_atributos_numericos   ),
    ('binarios',    'passthrough',          nomes_atributos_binarios    ),
    ('categoricos', OneHotEncoder(),        nomes_atributos_categoricos ),
    ],
    sparse_threshold=0)

# pre-processamento do conjunto de treino
X_treino = preproc_completo.fit_transform(df_treino)

# pre-processamento do conjunto de teste
X_teste = preproc_completo.transform(df_teste)
    
# Coluna alvo para os conjuntos de treino e teste
y_treino = df_treino_labels[ nome_coluna_alvo ] > 0
y_treino = y_treino.values.ravel()

y_teste  = df_teste_labels[  nome_coluna_alvo ] > 0
y_teste  = y_teste.values.ravel()

nomes_atributos = nomes_atributos_numericos
nomes_atributos = np.append( nomes_atributos, nomes_atributos_binarios )
nomes_atributos = np.append( nomes_atributos,
                             preproc_completo.named_transformers_['categoricos'].get_feature_names())

In [3]:
X_treino.shape, y_treino.shape

((43129, 43), (43129,))

In [4]:
y_treino

array([False, False, False, ..., False, False, False])

In [5]:
valores_y_treino, quantidades_y_treino = np.unique(y_treino, return_counts=True)
print(np.asarray((valores_y_treino, quantidades_y_treino)))
print("Proporção classe True: ",quantidades_y_treino[1]/y_treino.shape[0])

[[    0     1]
 [42995   134]]
Proporção classe True:  0.003106958195181896


In [6]:
X_teste.shape, y_teste.shape

((10783, 43), (10783,))

In [7]:
valores_y_teste, quantidades_y_teste = np.unique(y_teste, return_counts=True)
print(np.asarray((valores_y_teste, quantidades_y_teste)))
print("Proporção classe True: ",quantidades_y_teste[1]/y_teste.shape[0])

[[    0     1]
 [10740    43]]
Proporção classe True:  0.003987758508763795


In [8]:
nomes_atributos

array(['longitude', 'latitude', 'feridos', 'feridos_gr', 'mortes', 'auto',
       'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int',
       'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'queda_arr',
       'dia_bin', 'x0_ABALROAMENTO', 'x0_ATROPELAMENTO', 'x0_CAPOTAGEM',
       'x0_CHOQUE', 'x0_COLISÃO', 'x0_EVENTUAL', 'x0_INCÊNDIO',
       'x0_NAO CADASTRADO', 'x0_QUEDA', 'x0_TOMBAMENTO', 'x1_DOMINGO',
       'x1_QUARTA-FEIRA', 'x1_QUINTA-FEIRA', 'x1_SEGUNDA-FEIRA',
       'x1_SEXTA-FEIRA', 'x1_SÁBADO', 'x1_TERÇA-FEIRA',
       'x2_CENTRO              ', 'x2_LESTE               ',
       'x2_NORTE               ', 'x2_NÃO IDENTIFICADO    ',
       'x2_SUL                 ', 'x3_1', 'x3_5', 'x3_13'], dtype=object)

## Treinamento

### Treinamento de uma Regressão Logística com SKLearn

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [10]:
log_reg = LogisticRegression(max_iter=300)

log_reg.fit(X_treino, y_treino)
log_reg.classes_

array([False,  True])

In [11]:
y_previsto_reglog = log_reg.predict(X_teste)
y_previsto_reglog

array([False, False, False, ..., False, False, False])

In [12]:
print("Acurácia: ", accuracy_score(y_teste,y_previsto_reglog))

Acurácia:  0.9999072614300287


### Treinamento de uma Árvore de Decisão  com SKLearn

In [13]:
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()
arvore.fit(X_treino, y_treino)
arvore.classes_

array([False,  True])

In [14]:
y_previsto_arvore = arvore.predict(X_teste)
y_previsto_arvore

array([False, False, False, ..., False, False, False])

In [15]:
print("Acurácia: ", accuracy_score(y_teste,y_previsto_arvore))

Acurácia:  0.9995363071501437


# Avaliação de Desempenho

## Protocolo de validação cruzada
<p>A figura abaixo ilustra o funcionamento da validação cruzada:</p>
<p><a href="https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation">
    <img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" alt="Dimension levels.svg">
   </a></p>
   <p>Fonte: <a href="https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation">Documentação do SKLearn</a></p>
    


In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(arvore, X_treino, y_treino, cv=5)

print("Acurácia em cada particionamento: ",scores)
print("Acurácia média..................: ",scores.mean())

Acurácia em cada particionamento:  [0.99976814 0.99988407 0.99976814 1.         0.99988406]
Acurácia média..................:  0.999860883006213


## Otimizando os hiper-parâmetros

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = [
 {'max_depth':        [1,2,3,4,5,6,7,8,9,10],
  'min_samples_leaf': [2,3,4,5]
 }]

grid_search = GridSearchCV(arvore, param_grid)
grid_search.fit(X_treino, y_treino)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
   

In [18]:
grid_search.best_params_

{'max_depth': 2, 'min_samples_leaf': 2}

In [19]:
y_previsto_melhor_arvore = grid_search.best_estimator_.predict(X_teste)
print("Acurácia da melhor árvore: ", accuracy_score(y_teste,y_previsto_melhor_arvore))

Acurácia da melhor árvore:  0.9999072614300287


### Acurácia por um previsor ignorante

In [20]:
from sklearn.base import BaseEstimator

# implementando um previsor que sempre chuta na classe False, ignorando os valores em X
class PrevisorIgnorante(BaseEstimator):
    def fit(self, X, y=None):
        pass                                    # ou seja, não faz nada para aprender no método fit
    def predict(self, X):
        return np.zeros((len(X),), dtype=bool)  # ou seja, devolve um vetor de previsões "False", 
                                                # ignorando os valores dos atributos preditivos.

# treinando
ignorante = PrevisorIgnorante()
ignorante.fit(X_treino, y_treino)

In [21]:
y_previsto_ign = ignorante.predict(X_teste)
y_previsto_ign

array([False, False, False, ..., False, False, False])

In [22]:
print("Acurácia: ", accuracy_score(y_teste,y_previsto_ign))

Acurácia:  0.9960122414912362


## Melhorando a avaliação de desempenho

### Matriz de Confusão, também conhecida como Tabela de Contingência
<p>
Cada linha representa uma classe verdadeira, sendo a primeira a classe negativa e a segunda a classe positiva.<br>
Cada coluna representa uma classe prevista, sendo a primeira a classe negativa e a segunda a classe positiva.
</p>

In [23]:
print("                             ---------------------------")
print("                             |      Classe Prevista    |")
print("                             |  Negativa  |  Positiva  |")
print("--------------------------------------------------------")
print("                    Negativa |     VN     |     FP     |")
print("Classe Verdadeira            |-------------------------|")
print("                    Positiva |     FN     |     VP     |")
print("                             ---------------------------")

                             ---------------------------
                             |      Classe Prevista    |
                             |  Negativa  |  Positiva  |
--------------------------------------------------------
                    Negativa |     VN     |     FP     |
Classe Verdadeira            |-------------------------|
                    Positiva |     FN     |     VP     |
                             ---------------------------


In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score   # VP / (VP+FP)
from sklearn.metrics import recall_score      # VP / (VP+FN)
from sklearn.metrics import f1_score          # Média harmônica entre precisão e revocação (recall)

<p><b>Regressão logística:</b></p>

In [25]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_reglog))
print()
print("Precisão: ", precision_score( y_teste,y_previsto_reglog))
print("Recall  : ", recall_score(    y_teste,y_previsto_reglog))
print("F1      : ", f1_score(        y_teste,y_previsto_reglog))

Matriz de confusão:
[[10740     0]
 [    1    42]]

Precisão:  1.0
Recall  :  0.9767441860465116
F1      :  0.988235294117647


<p><b>Árvore de decisão:</b></p>

In [26]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_arvore))
print()
print("Precisão: ", precision_score( y_teste,y_previsto_arvore))
print("Recall  : ", recall_score(    y_teste,y_previsto_arvore))
print("F1      : ", f1_score(        y_teste,y_previsto_arvore))

Matriz de confusão:
[[10736     4]
 [    1    42]]

Precisão:  0.9130434782608695
Recall  :  0.9767441860465116
F1      :  0.9438202247191011


In [27]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_melhor_arvore))
print()
print("Precisão: ", precision_score( y_teste,y_previsto_melhor_arvore))
print("Recall  : ", recall_score(    y_teste,y_previsto_melhor_arvore))
print("F1      : ", f1_score(        y_teste,y_previsto_melhor_arvore))

Matriz de confusão:
[[10740     0]
 [    1    42]]

Precisão:  1.0
Recall  :  0.9767441860465116
F1      :  0.988235294117647


<p><b>Previsor ignorante:</b></p>

In [28]:
print("Matriz de confusão:")
print(confusion_matrix(y_teste,y_previsto_ign))
print()
print("Precisão: ", precision_score( y_teste,y_previsto_ign,  zero_division=0))
print("Recall  : ", recall_score(    y_teste,y_previsto_ign))
print("F1      : ", f1_score(        y_teste,y_previsto_ign))

Matriz de confusão:
[[10740     0]
 [   43     0]]

Precisão:  0.0
Recall  :  0.0
F1      :  0.0
